# ECE 269 - Linear Algebra - Fall 2019 - Final Project - EigenFaces

## EigenFaces project worked on by:-
## Name :- Anirudh Swaminathan
## PID :- A53316083
## Email ID :- aswamina@ucsd.edu

#### Notebook created by Anirudh Swaminathan from ECE department majoring in Intelligent Systems, Robotics and Control for the course ECE269 Linear Algebra for Fall 2019

In [ ]:
%matplotlib inline

# for working with images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# randomness
import random

# matrices
import numpy as np

# eigen value computation
from scipy.linalg import eigh

# for rotating images
from scipy import ndimage as ndi

In [ ]:
# dataset directory specified here
data_dir = "../datasets/"

In [ ]:
# data stuff here
neutral_imgs = []
smiling_imgs = []
mean_img = None
dataset = []
image_size = None
eigen_faces = None

### Load the data

In [ ]:
for i in range(200):
    # Read the neutral images from the (i+1)a.jpg file
    # Read the smiling images from the (i+1)b.jpg file
    nimg = mpimg.imread(data_dir + str(i + 1) + "a.jpg")
    simg = mpimg.imread(data_dir + str(i + 1) + "b.jpg")
    neutral_imgs.append(nimg)
    smiling_imgs.append(simg)
dataset = neutral_imgs[:190]

# Convert the dataset of 190 neutral images to a numpy array
dataset = np.array(dataset)

# note down the image size
image_size = (dataset.shape[1], dataset.shape[2])
dataset = dataset.reshape(dataset.shape[0], dataset.shape[1] * dataset.shape[2])
# 31266 * 190 - dataset shape
dataset = np.transpose(dataset)

In [ ]:
# convert the dataset to float to avoid errors in processing later on
dataset = dataset.astype(np.float64)

In [ ]:
# convert the neutral and smiling images dataset to Numpy
neutral_imgs = np.array(neutral_imgs)
smiling_imgs = np.array(smiling_imgs)

# convert these images to float to avoid errors in processing later on
neutral_imgs = neutral_imgs.astype(np.float64)
smiling_imgs = smiling_imgs.astype(np.float64)

### Display 5 random neutral and smiling images to verify if the data loading was correct

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(8, 8), sharex="all", sharey="all")
for i in range(5):
    j = random.randint(0, 199)
    n = neutral_imgs[j]
    s = smiling_imgs[j]
    
    # display the neutral image
    axes[i][0].imshow(n, cmap="gray")
    axes[i][0].axis('off')
    axes[i][0].set_title("Neutral Face")
    
    # display the smiling image
    axes[i][1].imshow(s, cmap="gray")
    axes[i][1].axis('off')
    axes[i][1].set_title("Smiling Face")
    
plt.tight_layout()
fig.canvas.draw()

### Question a) - Computing the PCs using 1st 190 individual's neutral expressions and plotting the singular values of the data matrix

#### Compute Mean Face

In [ ]:
# calculate the mean face for the dataset
num_imgs = dataset.shape[1]
mean_img = np.matmul(dataset, np.ones((num_imgs, 1))) / num_imgs
# 31266 * 1 - mean_img
mean_img = mean_img
print(mean_img.shape, mean_img.dtype, np.min(mean_img), np.max(mean_img))

In [ ]:
# plot the mean face of the neutral expressions
plt_mean_img = mean_img.reshape(image_size[0], image_size[1])
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(7, 6))
axes.imshow(plt_mean_img, cmap="gray")
axes.axis('off')
axes.set_title("Mean Face of 190 Neutral Faces")
fig.canvas.draw()

#### Compute the eigen values and the corresponding eigen faces

In [ ]:
# 31266 * 180
mean_offset = dataset - mean_img

# 180*180
mod_cov = np.matmul(np.transpose(mean_offset), mean_offset)

# check if this is a symmetric matrix
print(np.allclose(mod_cov, np.transpose(mod_cov)))

In [ ]:
# compute the eigen values and the corresponding eigenvectors in ascending order
# 190 eigen values
# 190 * 190 eigen vectors
# ith column - corresponding to the ith eigen vector
eig_vals, mod_eig_vecs = eigh(mod_cov)

# 31266 * 190 - eigen vectors of the original data
eig_vecs = np.matmul(mean_offset, mod_eig_vecs)

# normalize the eigen vectors now
# 31266 * 190 normalized eigen vectors
norm_cnst = np.sqrt(np.sum(np.square(eig_vecs), 0))
eigen_faces = np.divide(eig_vecs, norm_cnst)

In [ ]:
# flip both eigen values and eigen vectors to ensure the largest eigen vectors are at the front
eig_vals = np.flip(eig_vals)
eigen_faces = np.flip(eigen_faces, 1)

#### Plot the singular values of the data matrix

Since the matrix $A^TA$ is symmetric, it's singluar values are the square root of its eigen values, and they are real.

In [ ]:
# plot the singular values of the data matrix
indices = [i for i in range(190)]
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(7, 6))
axes.plot(indices, np.sqrt(eig_vals))
axes.set_xlabel("Indices")
axes.set_ylabel("Singular Values")
axes.set_xlim(-1, 200)
axes.set_ylim(0,)
axes.set_title("Singular Values of the Data Matrix")
fig.canvas.draw()

In [ ]:
# plot the eigen values of the data matrix
indices = [i for i in range(190)]
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(7, 6))
axes.plot(indices, eig_vals)
axes.set_xlabel("Indices")
axes.set_ylabel("Eigen Values")
axes.set_xlim(-1, 200)
axes.set_ylim(0, 1.75e9)
axes.set_title("Eigen Values of the Data Matrix")
fig.canvas.draw()

For choosing the eigen values, we take the largest eigen values as they indicate the maximum variance between the different features. We can choose the number of PCs as the one that gives >95% variance between the features.

In [ ]:
tot = np.sum(eig_vals)
su = 0
ans = -1
for j in range(190):
    su += eig_vals[j]
    if su/tot>0.95:
        ans = j+1
        print("The largest {} eigen vectors encode >95% variability of the dataset".format(ans))
        break

Hence, we can choose $96$ largest eigen values and their corresponding eigen vectors to represent greater than $95%$ variability of the given dataset of faces.

In [ ]:
# plot the top 5 eigen faces
fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(8, 8), sharex="all", sharey="all")
for i in range(5):
    img = eigen_faces[:, i]
    plot_face = img.reshape(image_size[0], image_size[1])
    
    # display the neutral image
    axes[i].imshow(plot_face, cmap="gray")
    axes[i].axis('off')
    axes[i].set_title("Eigen Face: {}".format(i+1))
    
plt.tight_layout()
fig.canvas.draw()

### Question b) - Reconstruction of a neutral individual's PCs using different number of PC's

In [ ]:
def reconstruct_image(orig, pcs):
    """A function to reconstruct the original image with the given PCs"""
    # perform mean subtraction
    # 31566 * 1
    #print("Original: {}, Mean Image: {}".format(orig.shape, mean_img.shape))
    mean_sub = orig - mean_img
    #print(mean_sub.shape)
    
    # calculate the weights
    # l * 31566 * 31566 * 1 = l * 1
    #print(pcs.shape)
    w = np.matmul(np.transpose(pcs), mean_sub)
    #print(w.shape)
    
    # 31566 * l * l * 1 = 31566 * 1
    recon = np.matmul(pcs, w) + mean_img
    #print(recon.shape)
    mse = np.mean(np.square(orig - recon))
    return recon, mse

In [ ]:
# random index to pick image from
rand_ind = random.randint(0, 189)
original_img = dataset[:, rand_ind]
#print(original_img.shape)
original_img = original_img[:, np.newaxis]
#print(original_img.shape)

In [ ]:
mses = []
recs = []

# Reconstruct the image for all the PCs
for j in range(190):
    #print(dataset.shape)
    pcs = eigen_faces[:, :(j+1)]
    #print(pcs.shape)
    rec, mse = reconstruct_image(original_img, pcs)
    recs.append(rec)
    mses.append(mse)

In [ ]:
print(len(mses), min(mses), max(mses))

In [ ]:
print(mses)

In [ ]:
# plot the singular values of the data matrix
num_eigs = [i+1 for i in range(190)]
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(7, 6))
axes.plot(num_eigs, mses)
axes.set_xlabel("Number of Largest Principal Components")
axes.set_ylabel("MSE of Reconstruction")
axes.set_xlim(-1, 200)
axes.set_ylim(0, )
#axes.set_title("Number of Principal Components vs Mean Squared Error of Reconstruction")
fig.canvas.draw()

In [ ]:
# Plot the reconstructed images for 5 different number of PCs
num_pcs = [1, 50, 100, 150, 189, 190]
fig, axes = plt.subplots(nrows=6, ncols=2, figsize=(8, 8), sharex="all", sharey="all")
for i in range(len(num_pcs)):
    r = recs[num_pcs[i] - 1]
    print(r.shape, r.dtype)
    r = r.reshape(image_size[0], image_size[1])
    
    # display the neutral image
    axes[i][0].imshow(original_img.reshape(image_size[0], image_size[1]), cmap="gray")
    axes[i][0].axis('off')
    axes[i][0].set_title("Original Face")
    
    # display the smiling image
    axes[i][1].imshow(r, cmap="gray")
    axes[i][1].axis('off')
    axes[i][1].set_title("Reconstructed Face using top {} eigen faces".format(num_pcs[i]))
    
plt.tight_layout()
fig.canvas.draw()

In [ ]:
r = recs[188]
print(r.shape, r.dtype, np.min(r), np.max(r))
print(original_img.shape, original_img.dtype, np.min(original_img), np.max(original_img))

$$\textbf{Qualitative Results}$$
I tried reconstruction of a random neutral image using different number of principal components with the largest eigen values <br>
I found that the reconstructed image was not close for just $1$ principal component. <br>
For more principal components, I found that the reconstructed image starts resembling the original image. <br>
$$\textbf{Quantitave Results}$$
As the number of principal components associated with the largest eigen values used to recreate the neutral image increases, the MSE for my model keeps on decreasing to $0$. This is to be expected as this neutral image was used in the computation of all the $190$ eigen faces, and hence, this neutral image can be spanned by all the $190$ principal components. <br>

### Question c) - Reconstruction of a smiling individual's PCs using different number of PC's

In [ ]:
# random index to pick image from
#rand_ind = random.randint(0, 189)
original_img = smiling_imgs[rand_ind, :, :]
print(original_img.shape)
original_img = original_img.reshape(image_size[0] * image_size[1], 1)
print(original_img.shape, original_img.dtype, np.min(original_img), np.max(original_img))

In [ ]:
sm_mses = []
sm_recs = []

# Reconstruct the image for all the PCs
for j in range(190):
    #print(dataset.shape)
    pcs = eigen_faces[:, :(j+1)]
    #print(pcs.shape)
    rec, mse = reconstruct_image(original_img, pcs)
    sm_recs.append(rec)
    sm_mses.append(mse)

In [ ]:
print(len(sm_mses), min(sm_mses), max(sm_mses))

In [ ]:
print(sm_mses)

In [ ]:
# plot the singular values of the data matrix
num_eigs = [i+1 for i in range(190)]
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(7, 6))
axes.plot(num_eigs, sm_mses)
axes.set_xlabel("Number of Largest Principal Components")
axes.set_ylabel("MSE of Reconstruction")
axes.set_xlim(-1, 200)
axes.set_ylim(0, )
#axes.set_title("Number of Principal Components vs Mean Squared Error of Reconstruction")
fig.canvas.draw()

In [ ]:
# Plot the reconstructed images for 5 different number of PCs
num_pcs = [1, 50, 100, 150, 189, 190]
fig, axes = plt.subplots(nrows=6, ncols=2, figsize=(8, 8), sharex="all", sharey="all")
for i in range(len(num_pcs)):
    r = sm_recs[num_pcs[i] - 1]
    print(r.shape, r.dtype)
    r = r.reshape(image_size[0], image_size[1])
    
    # display the neutral image
    axes[i][0].imshow(original_img.reshape(image_size[0], image_size[1]), cmap="gray")
    axes[i][0].axis('off')
    axes[i][0].set_title("Original Face")
    
    # display the smiling image
    axes[i][1].imshow(r, cmap="gray")
    axes[i][1].axis('off')
    axes[i][1].set_title("Reconstructed Face using top {} eigen faces".format(num_pcs[i]))
    
plt.tight_layout()
fig.canvas.draw()

In [ ]:
r = sm_recs[188]
print(r.shape, r.dtype, np.min(r), np.max(r))
print(original_img.shape, original_img.dtype, np.min(original_img), np.max(original_img))

$$\textbf{Qualitative Results}$$
I tried reconstruction of a random neutral image using different number of principal components with the largest eigen values <br>
I found that the reconstructed image was not close for just $1$ principal component. <br>
For more principal components, I found that the reconstructed image starts resembling the original image. <br>
$$\textbf{Quantitave Results}$$
As the number of principal components associated with the largest eigen values used to recreate the neutral image increases, the MSE for my model keeps on decreasing. This is to be expected, as the number of Pricipal components increases, they are able to approximately span the original image more and more. <br>
$$\textbf{Comparison with Neutral Image Reconstruction}$$
This reconstruction is not as good as the neutral images reconstruction. This is because the smiling image was not a part of process of finding the principal components. Hence, with the given principal components, we cannot span the smiling images.

### Question d) Reconstruction of test set of neutral images from the Principal Components

In [ ]:
# random index to pick image from
rand_neut = random.randint(190, 199)
original_img = neutral_imgs[rand_neut, :, :]
print(original_img.shape)
original_img = original_img.reshape(image_size[0] * image_size[1], 1)
print(original_img.shape, original_img.dtype, np.min(original_img), np.max(original_img))

In [ ]:
te_mses = []
te_recs = []

# Reconstruct the image for all the PCs
for j in range(190):
    #print(dataset.shape)
    pcs = eigen_faces[:, :(j+1)]
    #print(pcs.shape)
    rec, mse = reconstruct_image(original_img, pcs)
    te_recs.append(rec)
    te_mses.append(mse)

In [ ]:
print(len(te_mses), min(te_mses), max(te_mses))

In [ ]:
print(te_mses)

In [ ]:
# plot the singular values of the data matrix
num_eigs = [i+1 for i in range(190)]
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(7, 6))
axes.plot(num_eigs, te_mses)
axes.set_xlabel("Number of Largest Principal Components")
axes.set_ylabel("MSE of Reconstruction")
axes.set_xlim(-1, 200)
axes.set_ylim(0,)
#axes.set_title("Number of Principal Components vs Mean Squared Error of Reconstruction")
fig.canvas.draw()

In [ ]:
# Plot the reconstructed images for 5 different number of PCs
num_pcs = [1, 50, 100, 150, 189, 190]
fig, axes = plt.subplots(nrows=6, ncols=2, figsize=(8, 8), sharex="all", sharey="all")
for i in range(len(num_pcs)):
    r = te_recs[num_pcs[i] - 1]
    print(r.shape, r.dtype)
    r = r.reshape(image_size[0], image_size[1])
    
    # display the neutral image
    axes[i][0].imshow(original_img.reshape(image_size[0], image_size[1]), cmap="gray")
    axes[i][0].axis('off')
    axes[i][0].set_title("Original Face")
    
    # display the smiling image
    axes[i][1].imshow(r, cmap="gray")
    axes[i][1].axis('off')
    axes[i][1].set_title("Reconstructed Face using top {} eigen faces".format(num_pcs[i]))
    
plt.tight_layout()
fig.canvas.draw()

In [ ]:
r = sm_recs[188]
print(r.shape, r.dtype, np.min(r), np.max(r))
print(original_img.shape, original_img.dtype, np.min(original_img), np.max(original_img))

$$\textbf{Qualitative Results}$$
I tried reconstruction of a random neutral image using different number of principal components with the largest eigen values <br>
I found that the reconstructed image was not close for just $1$ principal component. <br>
For more principal components, I found that the reconstructed image starts resembling the original image. <br>
$$\textbf{Quantitave Results}$$
As the number of principal components associated with the largest eigen values used to recreate the neutral image increases, the MSE for my model keeps on decreasing. This is to be expected, as the number of Pricipal components increases, they are able to approximately span the original image more and more. <br>
$$\textbf{Comparison with Neutral Image Reconstruction}$$
This reconstruction is not as good as the neutral images reconstruction. This is because this neutral image was not a part of process of finding the principal components. Hence, with the given principal components, we cannot span this image.
$$\textbf{Comparison with Smiling Image Reconstruction}$$
This reconstruction is very similar to the one performed on smiling images, and the same argument applies. The fact that they do not contribute to finding the Principal Components leads to the fact that they cannot efficiently reconstruct the given image.

### Question e) Reconstruction of non-human image from the set of PCs

In [ ]:
# read car
car_pth = "../datasets/car.jpg"
car_img = mpimg.imread(car_pth)
car_img = car_img.astype(np.float64)
original_img = car_img.reshape(image_size[0] * image_size[1], 1)
print(original_img.shape, original_img.dtype, np.min(original_img), np.max(original_img))

In [ ]:
# Reconstruct the image for all the PCs
#print(dataset.shape)
pcs = eigen_faces
print(eigen_faces.shape)
#print(pcs.shape)
rec, mse = reconstruct_image(original_img, pcs)

In [ ]:
print(mse)

In [ ]:
# reconstruct for 189 PCs
pcs = eigen_faces[:, :189]
print(pcs.shape)
rec2, mse2 = reconstruct_image(original_img, pcs)

In [ ]:
print(mse2)

In [ ]:
# reconstruct for all 190 pcs
pcs = eigen_faces[:, :190]
print(pcs.shape)
rec3, mse3 = reconstruct_image(original_img, pcs)

In [ ]:
print(mse3)

In [ ]:
recs = [rec2, rec]

In [ ]:
# Plot the reconstructed images for 5 different number of PCs
num_pcs = [189, 190]
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(8, 8), sharex="all", sharey="all")
for i in range(len(num_pcs)):
    r = recs[i]
    print(r.shape, r.dtype)
    r = r.reshape(image_size[0], image_size[1])
    
    # display the neutral image
    axes[i][0].imshow(original_img.reshape(image_size[0], image_size[1]), cmap="gray")
    axes[i][0].axis('off')
    axes[i][0].set_title("Original Image")
    
    # display the smiling image
    axes[i][1].imshow(r, cmap="gray")
    axes[i][1].axis('off')
    axes[i][1].set_title("Reconstructed Image using top {} eigen faces".format(num_pcs[i]))
    
plt.tight_layout()
fig.canvas.draw()

In [ ]:
r = rec2
print(r.shape, r.dtype, np.min(r), np.max(r))
print(original_img.shape, original_img.dtype, np.min(original_img), np.max(original_img))

TODO

### Question f) Reconstruction of training set neutral image for different rotations

In [ ]:
original_img = dataset[:, rand_ind]
original_img = np.reshape(original_img, (image_size[0], image_size[1]))

In [ ]:
mses = []
rots = []
recs = []

pcs = eigen_faces[:, :189]

# Reconstruct the image for all the PCs
for j in range(361):
    #print(dataset.shape)
    rot_image = ndi.rotate(original_img, j, reshape=False)
    rot_image = rot_image.reshape(image_size[0]*image_size[1], 1)
    rots.append(rot_image)
    #print(rot_image.shape, rot_image.dtype)
    #print(pcs.shape)
    rec, mse = reconstruct_image(rot_image, pcs)
    recs.append(rec)
    mses.append(mse)

In [ ]:
print(len(mses), min(mses), max(mses))

In [ ]:
print(mses)

In [ ]:
# plot the singular values of the data matrix
angles = [i for i in range(361)]
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(7, 6))
axes.plot(angles, mses)
axes.set_xlabel("Angle of rotation in degrees")
axes.set_ylabel("MSE of Reconstruction")
axes.set_xlim(-1, 400)
axes.set_ylim(0, )
#axes.set_title("Number of Principal Components vs Mean Squared Error of Reconstruction")
fig.canvas.draw()

In [ ]:
# Plot the reconstructed images for 5 different number of PCs
angles = [0, 30, 60, 90, 120, 180, 240, 270, 300, 360]
fig, axes = plt.subplots(nrows=10, ncols=3, figsize=(12, 12), sharex="all", sharey="all")
for i in range(len(angles)):
    r = recs[angles[i]]
    print(r.shape, r.dtype)
    r = r.reshape(image_size[0], image_size[1])
    
    ro = rots[angles[i]]
    print(ro.shape, ro.dtype)
    ro = ro.reshape(image_size[0], image_size[1])
    
    # display the neutral image
    axes[i][0].imshow(original_img.reshape(image_size[0], image_size[1]), cmap="gray")
    axes[i][0].axis('off')
    axes[i][0].set_title("Original Face")
    
    # display the rotated image
    axes[i][1].imshow(ro, cmap="gray")
    axes[i][1].axis('off')
    axes[i][1].set_title("Rotated Image Angle: {}".format(angles[i]))
    
    # display the smiling image
    axes[i][2].imshow(r, cmap="gray")
    axes[i][2].axis('off')
    axes[i][2].set_title("Reconstructed Face with 189 PCs".format(angles[i]))
    
plt.tight_layout()
fig.canvas.draw()

In [ ]:
mses = []
rots = []
recs = []

pcs = eigen_faces[:, :190]

# Reconstruct the image for all the PCs
for j in range(361):
    #print(dataset.shape)
    rot_image = ndi.rotate(original_img, j, reshape=False)
    rot_image = rot_image.reshape(image_size[0]*image_size[1], 1)
    rots.append(rot_image)
    #print(rot_image.shape, rot_image.dtype)
    #print(pcs.shape)
    rec, mse = reconstruct_image(rot_image, pcs)
    recs.append(rec)
    mses.append(mse)

In [ ]:
print(len(mses), min(mses), max(mses))

In [ ]:
print(mses)

In [ ]:
# plot the singular values of the data matrix
angles = [i for i in range(361)]
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(7, 6))
axes.plot(angles, mses)
axes.set_xlabel("Angle of rotation in degrees")
axes.set_ylabel("MSE of Reconstruction")
axes.set_xlim(-5, 400)
axes.set_ylim(0, )
#axes.set_title("Number of Principal Components vs Mean Squared Error of Reconstruction")
fig.canvas.draw()

In [ ]:
# Plot the reconstructed images for 5 different number of PCs
angles = [0, 30, 60, 90, 120, 180, 240, 270, 300, 360]
fig, axes = plt.subplots(nrows=10, ncols=3, figsize=(12, 12), sharex="all", sharey="all")
for i in range(len(angles)):
    r = recs[angles[i]]
    print(r.shape, r.dtype)
    r = r.reshape(image_size[0], image_size[1])
    
    ro = rots[angles[i]]
    print(ro.shape, ro.dtype)
    ro = ro.reshape(image_size[0], image_size[1])
    
    # display the neutral image
    axes[i][0].imshow(original_img.reshape(image_size[0], image_size[1]), cmap="gray")
    axes[i][0].axis('off')
    axes[i][0].set_title("Original Face")
    
    # display the rotated image
    axes[i][1].imshow(ro, cmap="gray")
    axes[i][1].axis('off')
    axes[i][1].set_title("Rotated Image Angle: {}".format(angles[i]))
    
    # display the smiling image
    axes[i][2].imshow(r, cmap="gray")
    axes[i][2].axis('off')
    axes[i][2].set_title("Reconstructed Face with all PCs".format(angles[i]))
    
plt.tight_layout()
fig.canvas.draw()

TODO

TODO - Final answers to stuff, conclusions, math